# Nonlinear Equations

## 05.01 Nonlinear Equations

Given a function f, we seek a value $x$, known as **root** or **zero** of f, for which
$$
f(x) = 0
$$

Two cases.
1. Single nonlinear equation in one unknown, $x$ is a scalar $f: \mathbb{R} \rightarrow \mathbb{R}$

Example:
$$
x^2 - 4 sin(x) = 0
$$

2. System of $n$ coupled nonlinear equations in $n$ unknowns, $x$ is vector
$f: \mathbb{R}^n \rightarrow \mathbb{R}^n$

Example:
$$
\begin{aligned}
x_1^2 - x_2 + 0.25 &= 0 \\
-x_1 + x_2^2 + 0.25 &= 0
\end{aligned}
$$

Note: Finding $f(x) = 0$ for overdetermined systems, $m \gt n$, is covered by the chapter on optimization. 

## 05.02 Existence, Uniqueness, and Conditioning

**Bracket** is interval \[a, b\] for which sign of $f$ differs at $f(a)$ and $f(b)$.
* *Intermediate Value Theorem* if $f$ is continuous and sign($f(a)$) $\neq$ sign($f(b)$), then there exists some $x$ such that f(x) = 0.

#### Conditioning
Basic premise: If the function $f$ is insensitive to the value of the argument $x$, then the root will be sensitive.

Two cases.
1. $f: \mathbb{R} \rightarrow \mathbb{R}$ with solution $x^*$
$$
\text{cond}(f) = 1 / f'(x^*)
$$
  * Root is ill-conditioned if $f'(x^*) = 0$ eg horizontal.

2. $f: \mathbb{R}^n \rightarrow \mathbb{R}^n$ with solution $x^*$
$$
\text{cond}(f) = ||J_f^{-1}(x^*)|| = \partial{f_i(x)} / \partial{x_j}
$$
  * The matrix $J_f$ is known as **Jacobian** aka matrix of partial derivatives.
  * Root is ill-conditioned if $J_f$ is singular.

#### Residual
Useful as measure of error when root is not ill-conditioned.

A small residual does not necessarily imply solution is accurate.

## 05.03 Convergence of Iterative Methods

#### Iterative Methods
**Error** at iteration $k$ is given by $e_k$:
$$
e_k = x_k - x^*
$$
where
* $x_k$ is approximate solution at iteration $k$
  * For methods which maintain an interval rather than single point, then $x_k$ is the length of the interval and $x^*$ is zero.
* $x^*$ is true solution

**Convergence** given by some rate $r$:
$$
\lim\limits_{k->\inf} \frac{||e_{k+1}||}{||e_k||^r} = C
$$
where
* $C$ is a nonzero constant describing the change from one iteration to the next

|  convergence rate $r$ | description | accuracy gained per iteration |
|-----------------------|-------------|-------------------------------|
| $r = 1$ | linear | constant |
| $1 \lt r \lt 2$ | superlinear | increasing |
| $r = 2$ | quadratic | double |

**Stopping Criteria** describe conditions for terminating iteration. Typical choices include:
1. Relative change in successive iterates is small.
$$
||x_{k+1} - x_k|| / ||x_k|| \lt \epsilon
$$
2. Residual is small.
$$
||f(x_k)|| \lt \epsilon
$$

These quantities are not necessarily small simultaneously depending on the problem conditioning.

## 05.04 Bisection Method in 1D

Bisection method begins with bracket given by $[a, b]$, repeatedly halving the interval until solution is found to some particular accuracy.
1. Compute the value of $f$ at midpoint $m$ of the bracket $[a, b]$.
2. If sign(f(a)) $\neq$ sign(f(m)), then $b = m$, else $a = m$.
3. Repeat while $b - a \gt \epsilon$.

Pros
* Certain to converge
* Simple, only makes use of sign of function, not magnitude

Cons
* Convergence is linear ($r = 1, C = 0.5$) which is slow
  * One bit of accuracy gained for each iteration

Find the root of the scalar function $x^2 - 4 \sin(x) = 0$ using bisection method.

In [1]:
import math
import numpy as np

def bisection1d(fx, a, b, eps=np.finfo('d').eps):
    """
    Compute the root of fx over some interval [a, b].
    """
    while abs(b - a) > eps:
        m = a + (b - a) / 2.
        if np.sign(fx(a)) != np.sign(fx(m)):
            b = m
        else:
            a = m
    return a


def fx(x):
    return x*x - 4. * math.sin(x)


root = bisection1d(fx, a=1., b=3.)
print("root: ", root)

# Compare the value at root to 0.
np.testing.assert_almost_equal(fx(root), 0.)

root:  1.9337537628270212


## 05.05 Fixed-Point Iteration in 1D

**Fixed point** of a function $g$ is a value $x$ such that $x = g(x)$.
* Iterative methods for nonlinear equations use fixed-point scheme:
$$
x_{k+1} = g(x_k)
$$
* For some nonlinear equation $f(x) = 0$ there can be **multiple equivalent** choices for $g$.

Examples, let $f(x) = x^2 - x - 2$ so that $f(2) = 0$
1. $g(x) = x^2 - 2$ and $g(2) = 2$
2. $g(x) = 1 + 2/x$ and $g(2) = 2$
3. $g(x) = \sqrt{x + 2}$ and $g(2) = 2$

## 05.06 Newton's Method in 1D

#### Truncated Taylor Series
The function $f(x + h)$ is a linear function of $h$ that approximates $f$ near $x$.
$$
f(x + h) \approxeq f(x) + f'(x) h
$$

#### Newton's Method
1. Start with some initial guess $x_0$.
2. Compute $x_{k+1} = x_k - \frac{f(x_k)}{f'(x_k)}$.
3. Repeat the previous step until some stopping criteria is reached.


**Convergence**

Transform $f(x) = 0$ to the fixed point problem $g(x) = x - f(x) / f'(x)$.
* Convergence rate is quadratic $(r = 2)$.
* The initial guess, $x_0$, must be close to $f(x) = 0$ in order to converge.
* Unlike bisection, Newton's method is **not** guaranteed to converge.

Find the root of scalar function $x^2 - 4 \sin(x) = 0$ using Newton's method.

In [2]:
import math
import numpy as np

def newton1d(fx, dfx, x0, eps=np.finfo('d').eps):
    """
    Compute the root of fx near x0.
    """
    xk, fxk, hk = x0, 1., 1.
    # Stop iteration when hk = 0 or fx = 0.
    while abs(hk)/abs(xk) > eps or abs(fxk) > eps:
        fxk = fx(xk)
        hk = -1. * fxk / dfx(xk)
        xk = xk + hk
    return xk


def fx(x):
    return x*x - 4. * math.sin(x)

def dfx(x):
    return 2.*x - 4. * math.cos(x)


root = newton1d(fx, dfx, x0=3., eps=1e-7)
print("root: ", root)

# Compare the value at root to 0.
np.testing.assert_almost_equal(fx(root), 0.)

root:  1.9337537628270212
